In [1]:
import pandas as pd
import numpy as np
import sqlite3
from collections import defaultdict
import math
import re
from itertools import combinations 
import unidecode

In [2]:
con = sqlite3.connect('/Users/qlanners/projects/mlb_salary_comp_db/mlb_salary_comp.db')

In [3]:
inj_orig = pd.read_csv('../data/missedGamesInjuries.csv')

In [4]:
dl_orig = pd.read_csv('../data/missedGamesDL.csv')

In [5]:
# match full team names to abbreviations
teams = {
    'Angels': 'LAA',
    'Diamondbacks': 'ARI',
    'Braves': 'ATL',
    'Orioles': 'BAL',
    'Red Sox': 'BOS',
    'Cubs': 'CHC',
    'White Sox': 'CHW',
    'Reds': 'CIN',
    'Indians': 'CLE',
    'Rockies': 'COL',
    'Tigers': 'DET',
    'Marlins': 'MIA',
    'Astros': 'HOU',
    'Royals': 'KCR',
    'Dodgers': 'LAD',
    'Brewers': 'MIL',
    'Twins': 'MIN',
    'Expos': 'MON',
    'Mets': 'NYM',
    'Yankees': 'NYY',
    'Athletics': 'OAK',
    'Phillies': 'PHI',
    'Pirates': 'PIT',
    'Padres': 'SDP',
    'Mariners': 'SEA',
    'Giants': 'SFG',
    'Cardinals': 'STL',
    'Rays': 'TBR',
    'Devil Rays': 'TBR',
    'Rangers': 'TEX',
    'Blue Jays': 'TOR',
    'Nationals': 'WSN'
}

In [6]:
inj_updated = []
for p in list(set(inj_orig[inj_orig['Relinquished'].notnull()]['Relinquished'])):
    inj_player = inj_orig[(inj_orig['Relinquished'] == p) | (inj_orig['Acquired'] == p)].reset_index()
    inj_player_len = inj_player.shape[0]
    for i,v in inj_player.iterrows():
        if isinstance(v['Relinquished'], str):
            if inj_player_len > i+1:
                next_row = inj_player.iloc[i+1]
                if isinstance(next_row['Acquired'], str) and (v['Date'].split('-')[0] in next_row['Date']):
                    inj_updated.append([p, v['Date'], v['Team'], next_row['Date'], next_row['Team'], v['Notes']])
                else:
                    inj_updated.append([p, v['Date'], v['Team'], None, None, v['Notes']])
            else:
                inj_updated.append([p, v['Date'], v['Team'], None, None, v['Notes']])
                

In [7]:
dl_updated = []
for p in list(set(dl_orig[dl_orig['Relinquished'].notnull()]['Relinquished'])):
    dl_player = dl_orig[(dl_orig['Relinquished'] == p) | (dl_orig['Acquired'] == p)].reset_index()
    dl_player_len = dl_player.shape[0]
    for i,v in dl_player.iterrows():
        if isinstance(v['Relinquished'], str):
            if dl_player_len > i+1:
                next_row = dl_player.iloc[i+1]
                if isinstance(next_row['Acquired'], str) and (v['Date'].split('-')[0] in next_row['Date']):
                    dl_updated.append([p, v['Date'], v['Team'], next_row['Date'], next_row['Team'], v['Notes']])
                else:
                    dl_updated.append([p, v['Date'], v['Team'], None, None, v['Notes']])
            else:
                dl_updated.append([p, v['Date'], v['Team'], None, None, v['Notes']])
                

In [8]:
col_names = ['name', 'injuryStart', 'injStartTeam', 'injEnd', 'injEndTeam', 'injNotes']
inj_updated_df = pd.DataFrame(inj_updated, columns=col_names)
dl_updated_df = pd.DataFrame(dl_updated, columns=col_names)

# add column to keep the rows seperate for final loads into tables
inj_updated_df['table'] = 'injury'
dl_updated_df['table'] = 'dl'

# concat df together to process name keys on both at same time
inj_updated_df = pd.concat([inj_updated_df, dl_updated_df])

# for some reason, one row is Nationals for name...
inj_updated_df = inj_updated_df[inj_updated_df['name'] != 'Nationals']

In [9]:
inj_updated_df = inj_updated_df.replace(np.nan, '', regex=True)
inj_updated_df['injStartTeam'] = [teams[t] if t else None for t in inj_updated_df['injStartTeam']]
inj_updated_df['injEndTeam'] = [teams[t] if t else None for t in inj_updated_df['injEndTeam']]

In [10]:
name_to_id = {n:None for n in inj_updated_df['name'].unique()}
total_players = len(name_to_id)

In [11]:
cur = con.cursor()

for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        year = int(r.injuryStart.split("-")[0])
        cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%" and year == {year}')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]

unmatched_round_1 = [n for n,v in name_to_id.items() if v is None]
name_team_year_matches = len([n for n,v in name_to_id.items() if v is not None])
print(f'{name_team_year_matches} of {total_players} matches on name, team, and year. ({round((name_team_year_matches / total_players)*100, 2)}%)')


3293 of 3648 matches on name, team, and year. (90.27%)


In [12]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%"')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%"')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_2 = [n for n,v in name_to_id.items() if v is None]
name_team_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_1])
print(f'{name_team_matches} of {total_players} matches on name and team. ({round((name_team_matches / total_players)*100, 2)}%)')
print('Players matched w/o year:')
print([p for p in unmatched_round_1 if p not in unmatched_round_2])
                  

136 of 3648 matches on name and team. (3.73%)
Players matched w/o year:
['Casey Kelly', 'Carl Sadler', 'Roy Alvarez / R.J. Alvarez', 'Calvin Pickering', 'Taylor Green', 'Kevin Hart', 'Chris Spurling', 'Brandon Workman', 'Kyle Denney', 'Jose Ceda', 'Logan Darnell', 'Chris Pettit', 'Andrew McKirahan', 'Juan Padilla', 'Andrew Brackman', 'Nick Rumbelow', 'Brandon Cumpton', 'Jeff Zimmerman', 'Adam Plutko', 'Colin Curtis', 'Erik Cordier', 'Dave Gassner', 'Jordan Smith (L.)', 'Craig Dingman', 'Davis Romero', 'Walter Majewski / Val Majewski', 'Dave Coggin', 'Gene Stechschulte', 'Dirk Hayhurst', 'Troy Mattes', 'Jose Torres (Marcos)', 'Austin Hays', 'John Ennis', 'Austin Maddox', 'Aaron Rakers', 'Jacob Nix', 'Jorge Luis Toca / Jorge Toca', 'Ian Hamilton', 'Ryan Merritt', 'Joseph Ortiz (b) / Joe Ortiz (b)', 'Frank Herrmann', 'Ryan Feierabend', 'Trevor Oaks', 'Eric Cammack', 'Engel Beltre', 'Jordan Tata', 'Jason Shiell', 'Matt Antonelli', 'Matt Chico', 'Francisco Cordova', 'Mario Valdez', 'Dave Ma

In [13]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        year = int(r.injuryStart.split("-")[0])
        cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and year == {year}')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_3 = [n for n,v in name_to_id.items() if v is None]
name_year_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_2])
print(f'{name_year_matches} of {total_players} matches on name and year. ({round((name_year_matches / total_players)*100, 2)}%)')
print('Players matched w/o team:')
print([p for p in unmatched_round_2 if p not in unmatched_round_3])

12 of 3648 matches on name and year. (0.33%)
Players matched w/o team:
['Nick Green (Alvin)', 'Ramon Cabrera', 'Ramon Laureano', 'Jose Reyes (Simon)', 'Michael Taylor (Anthony)', 'Jake Elmore', 'Kimera Bartee', 'Bobby Jones (Mitchell)', 'Brad Voyles', 'Ryan Budde', 'Brian Lawrence', 'Josh Johnson (Rashaad)']


In [14]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name_parts = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip().split()
        for p in combinations(name_parts, 2):
            name = ' '.join(p)
            year = int(r.injuryStart.split("-")[0])
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]
                break
    if not name_to_id[r['name']] and '/' in r['name']:
        name_parts = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip().split()
        for p in combinations(name_parts, 2):        
            name = ' '.join(p)
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]
                break

unmatched_round_4 = [n for n,v in name_to_id.items() if v is None]
part_name_team_year_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_3])
print(f'{part_name_team_year_matches} of {total_players} matches on partial-name, team, and year. ({round((part_name_team_year_matches / total_players)*100, 2)}%)')
print('Players matched on part of name:')
print([p for p in unmatched_round_3 if p not in unmatched_round_4])


33 of 3648 matches on partial-name, team, and year. (0.9%)
Players matched on part of name:
['Damaso Marte Sr.', 'Jose Vizcaino Sr.', 'Pedro Florimon Jr.', 'Jerry Hairston Jr.', 'Tony Gwynn Sr.', 'Gary Matthews Jr.', 'Delino DeShields Jr.', 'Tony Gwynn Jr. / Anthony Gwynn', 'John Mayberry Jr.', 'Roberto Hernandez Heredia', 'Torii Hunter Sr.', 'Albert Almora Jr.', 'Fernando Tatis Sr.', 'Ken Griffey Sr.', 'Vladimir Guerrero Sr.', 'Jose Cruz Jr.', 'Sandy Alomar Jr.', 'Tony Armas Jr.', 'Cesar Izturis Sr.', 'Rickie Weeks Jr.', 'Ivan DeJesus Jr. / Ivan De Jesus Jr.', 'Tony Pena Jr. (F.)', 'Shawon Dunston Sr.', 'Dave Berg (a)?', 'Benito Santiago Sr.', 'Eric Young (a)?', 'Robbie Ross Jr.', 'Brian Hunter (L.) or Brian Hunter (R.)', 'Tim Raines Sr.', 'Glenallen Hill Sr.', 'Delino DeShields Sr.', 'LaMonte Wade Jr.', 'Ed Sprague Jr.']


In [15]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = unidecode.unidecode(re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip().replace('.',''))
        year = int(r.injuryStart.split("-")[0])
        cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%" and year == {year}')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = unidecode.unidecode(re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip().replace('.',''))
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]

unmatched_round_5 = [n for n,v in name_to_id.items() if v is None]
clean_name_team_year_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_4])
print(f'{clean_name_team_year_matches} of {total_players} matches on cleaned-name, team, and year. ({round((clean_name_team_year_matches / total_players)*100, 2)}%)')
print('Players matched on unicode cleaned name:')
print([p for p in unmatched_round_4 if p not in unmatched_round_5])


13 of 3648 matches on cleaned-name, team, and year. (0.36%)
Players matched on unicode cleaned name:
['C.C. Sabathia', 'Carlos Corporán', 'Héctor Rondón', 'D.J. LeMahieu', 'A.J. Ramos', 'Joshua Riddle / J.T. Riddle', 'J.B. Shuck', 'Eduardo Núñez', 'J.T. Chargois', 'D.J. Stewart', 'A.J. Hinch', 'J.C. Ramirez', 'Vidal Nuño']


In [16]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%"')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%"')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_6 = [n for n,v in name_to_id.items() if v is None]
name_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_5])
print(f'{name_matches} of {total_players} matches on only name. ({round((name_matches / total_players)*100, 2)}%)')
print('Players matched on only name:')
print([p for p in unmatched_round_5 if p not in unmatched_round_6])

41 of 3648 matches on only name. (1.12%)
Players matched on only name:
['Matt Purke', 'Reynaldo Garcia', 'Mauricio Robles', 'Juan Gonzalez', 'Antoan Richardson', 'Burch Smith', 'Kyle Skipworth', 'Thomas Diamond', 'Francisco Peguero', 'Jose De Paula', 'Alejandro Machado', 'Steve Torrealba', 'Michael Coleman', 'Rymer Liriano', 'Rafael Lopez (Manuel) / Raffy Lopez', 'Taylor Guerrieri', 'Tim Wood', 'Brian Moran', 'Justin Williams (Paul)', 'Mike Sirotka', 'Aaron Crow', 'Jamie Brown', 'Elvin Ramirez', 'Tim Federowicz', 'Aaron Brooks (Lee)', 'Ronald Herrera', 'Michael Schwimer', 'German Duran', 'Mark Worrell', 'Brett Marshall', 'Anderson Garcia', 'Henry Sosa', 'Jose Campos (Vicente) / Vicente Campos', 'Dayan Diaz', 'Alfredo Gonzalez (a)', 'Josh Wilson (Aaron)', 'Jarrett Hoffpauir', 'Nik Turley', 'Taylor Thompson', 'Brad Snyder', 'Steve Schmoll']


In [17]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip().replace(' Jr.','').split()[-1]
        cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%"')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip().replace(' Jr.','').split()[-1]
            cur.execute(f'SELECT DISTINCT(nameKey) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.injStartTeam}%"')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_7 = [n for n,v in name_to_id.items() if v is None]
last_name_team_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_6])
print(f'{last_name_team_matches} of {total_players} matches on only last name and team. ({round((last_name_team_matches / total_players)*100, 2)}%)')
print('Players matched on last name and team:')
print([p for p in unmatched_round_6 if p not in unmatched_round_7])

22 of 3648 matches on only last name and team. (0.6%)
Players matched on last name and team:
['Jake Blalock', 'Jonathon Niese', 'Matt Joyce', 'Hong-Chih Kuo', 'Francisco Rosario', 'Samuel Deduno', 'Steve Tolleson', 'Jean Rodriguez', 'Jae-Weong Seo / Jae Seo', 'Zack Jones', 'Ken Roberts (b)', 'Gustavo Nunez', 'Andrew Sisco', 'Ismael Valdes', 'Dail Villanueva', 'Jesus Sosa', 'Todd Worrell', 'Jae-Kuk Ryu', 'Nate Adcock', 'Curtis Leskanic', 'Elmer Reyes', 'Terrell Young']


In [18]:
print(f'{len(unmatched_round_7)} players left unmatched')
print('Unmatched players: ')
num = 1
for p in unmatched_round_7:
    print(f'\t{num}. {p}')
    num += 1

98 players left unmatched
Unmatched players: 
	1. Alan Farina
	2. Jason Stumm
	3. Kris Honel
	4. Roberto Hernandez
	5. Alfredo Silverio
	6. Jose Oquendo
	7. Mark Merila
	8. Jon Shave
	9. Zach Britton
	10. Dave Williams (Aaron)
	11. John Farrell (Edward)
	12. Tony Cloninger
	13. David Montgomery
	14. Al Reyes
	15. Jason Kipnish
	16. Joe Maddon
	17. John Coppolella
	18. Ryan Westmoreland
	19. Ben Copeland
	20. Jose Mujica (Angel)
	21. Carlos Guzman (Manuel) (a)
	22. Armando Rivero
	23. Joan Gregorio
	24. Derrick Lewis
	25. Henry Barrera
	26. Brad Meyers
	27. Alex Wojchiechowski
	28. Ryan Ketchner
	29. Luis Urguetto
	30. Bobby Jones (Joseph)
	31. Francelis Montas / Francellis Montas / Frankie Montas
	32. Juan Pena
	33. Mike Sirtoka
	34. Jose Marmolejos-Diaz
	35. Brian Corey
	36. Chris Lee (Ulysses)
	37. Ramon Pena (Antonio)
	38. Kelvin Pichardo
	39. Eric Beaulac
	40. Riley Ferrell
	41. placed on 10-day IL with torn ligament in left middle finger
	42. Julian Fernandez
	43. sprained left th

In [19]:
inj_updated_df.shape

(22964, 7)

In [20]:
inj_df = inj_updated_df[~inj_updated_df['name'].isin(unmatched_round_7)]
print(f'Dropped rows for unmatched players\nCut DF from {inj_updated_df.shape[0]} to {inj_df.shape[0]} rows. (-{inj_updated_df.shape[0]-inj_df.shape[0]})')

Dropped rows for unmatched players
Cut DF from 22964 to 22818 rows. (-146)


In [21]:
inj_df['nameKey'] = ""
for i,r in inj_df.iterrows():
    r['nameKey'] = name_to_id[r['name']]

/Users/qlanners/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
dl_df_final = inj_df[inj_df['table'] == 'dl'].drop(columns=['table'])[['name', 'nameKey', 'injuryStart', 'injStartTeam', 'injEnd', 'injEndTeam', 'injNotes']]
inj_df_final = inj_df[inj_df['table'] == 'injury'].drop(columns=['table'])[['name', 'nameKey', 'injuryStart', 'injStartTeam', 'injEnd', 'injEndTeam', 'injNotes']]

In [29]:
print(f'Disabled List DF Rows: {dl_df_final.shape[0]}')
print(f'Injury List DF Rows: {inj_df_final.shape[0]}')
print(f'Total Rows between tables: {dl_df_final.shape[0] + inj_df_final.shape[0]}')

Disabled List DF Rows: 11812
Injury List DF Rows: 11006
Total Rows between tables: 22818


In [31]:
dl_df_final.head()

,name,nameKey,injuryStart,injStartTeam,injEnd,injEndTeam,injNotes
0,Billy McMillon,mcmilbi01,2001-07-26,OAK,,None,placed on 15-day DL with right shoulder inflam...
1,Billy McMillon,mcmilbi01,2001-09-11,OAK,,None,transferred to 60-day DL
2,Billy McMillon,mcmilbi01,2004-06-17,OAK,2004-08-11,OAK,placed on 15-day DL with lower back inflammation
3,Aaron Heilman,heilmaa01,2011-04-15,ARI,2011-05-05,ARI,placed on 15-day DL with right shoulder tendin...
4,Javier Baez,baezja01,2016-04-03,CHC,2016-04-15,CHC,placed on 15-day DL with bruised left thumb


In [ ]:
dl_df.to_sql('pst_disab', con, index=False)
inj_df.to_sql('bba_salaries', con, index=False)

In [ ]:
print('Job complete')